In [1]:
import pandas as pd 
import numpy as np

In [4]:
import os
os.chdir("/ada/projects/venia2021")

In [9]:
meme_vec = pd.read_csv("Rick Astley-vector_f2_2.csv")

In [11]:
data = pd.read_csv("reddit_methods/data/url_sample.csv")

In [30]:
user_counts = data.groupby(['author','domain'])['id'].count()

In [16]:
user_counts = user_counts[user_counts < 20].reset_index()

In [18]:
user_counts['key'] = user_counts['author'] + '_' + user_counts['domain']

In [24]:
user_counts['count'] = user_counts['id']

In [20]:
data['key'] = data['author'] + '_' + data['domain']

In [26]:
data2 = pd.merge(user_counts[['key','count']], data, on ='key')

In [53]:
metadata = pd.read_csv("data/reddit-master-metadata.tsv", sep='\t', )
vecs = pd.read_csv("data/reddit-master-vectors.tsv", sep ='\t', names =list(np.arange(150)))

reddit_embedding = pd.merge(metadata['community'],vecs,left_index=True,right_index=True)

In [57]:
reddit_embedding['subreddit'] = reddit_embedding['community'] 
reddit_embedding.drop('community',axis=1,inplace=True)

In [89]:
entity_sub = data2.groupby(['domain','subreddit'])['id'].count().reset_index()
entity_counts = data2.groupby('domain')['id'].count()
entity_counts = entity_counts[entity_counts> 100].reset_index()

entity_sub = pd.merge(entity_sub, reddit_embedding, on='subreddit')
entity_sub['count'] = entity_sub['id']
entity_sub.drop(['id'], axis=1,inplace=True)
entity_sub_count = pd.merge(entity_sub, entity_counts, on = 'domain')

for i in range(150):
    entity_sub_count[i] = entity_sub_count[i] * entity_sub_count['count'] / entity_sub_count['id']
    
entity_embedding = entity_sub_count.groupby('domain')[np.arange(150)].sum()

temp  =entity_embedding[entity_embedding.index.isin(data2.groupby('domain')['id'].count().sort_values(ascending=False).head(50).index)]

entity_embedding['sum'] = (entity_embedding[np.arange(150)] * entity_embedding[np.arange(150)]).sum(axis=1) ** .5

for i in range(150):
    entity_embedding[i] = entity_embedding[i] / entity_embedding['sum']
entity_embedding.drop('sum',axis=1,inplace=True)

In [90]:
from sklearn.metrics import pairwise

sim = pairwise.cosine_similarity(entity_embedding.values, meme_vec.T.values)

In [91]:
sim_df = pd.DataFrame(sim, index = entity_embedding.index, columns = ['cos_sim']).sort_values(by='cos_sim', ascending=False)

In [92]:
sim_df.tail(40)

,cos_sim
domain,
nola.com,-0.274949
lulus.com,-0.275201
native-instruments.com,-0.275686
thescore.com,-0.275854
jacobinmag.com,-0.276997
redfin.com,-0.277389
drmartens.com,-0.279082
fragrantica.com,-0.279833
llbean.com,-0.280098
